In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt 


In [3]:
IMAGE_SIZE = 256
BATCH_SIZE = 32
CHANNELS = 3 
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    'data', 
    shuffle = True,
    image_size= (IMAGE_SIZE, IMAGE_SIZE),
    batch_size= BATCH_SIZE
    
)

Found 2103 files belonging to 3 classes.


In [4]:
class_names = dataset.class_names
class_names

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

In [4]:
for image_batch, label_batch in dataset.take(1):
    print(image_batch.shape)
    print(label_batch.numpy)

(32, 256, 256, 3)
<bound method _EagerTensorBase.numpy of <tf.Tensor: shape=(32,), dtype=int32, numpy=
array([0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 2, 0, 0, 0, 2, 1, 0, 1, 1, 1, 0,
       1, 2, 1, 1, 0, 1, 0, 1, 0, 0])>>


In [ ]:
for image_batch, label_batch in dataset.take(1):
    plt.imshow(image_batch[0].numpy().astype('uint8'))
    plt.title(class_names[label_batch[0]])
    plt.axis('off')
    

In [4]:
# Training set 80%
# Validation set 10%
# Testing set 10%

training_set = 0.80
len(dataset)* training_set


52.800000000000004

In [5]:
train_ds = dataset.take(53) # Takes in the first 53 images as training data
test_ds = dataset.skip(53)
len(test_ds)

13

In [ ]:
val_size = 0.10
len(dataset)* val_size

In [ ]:
val_ds = test_ds.take(6)
len(val_ds)

In [ ]:
test_ds = val_ds.skip(6)
len(test_ds)

In [6]:
def get_dataset_split(df, train_split = 0.8, val_split = 0.1, test_split = 0.1, shuffle = True, shuffle_size = 10000):
    
    if shuffle:
        df = df.shuffle(shuffle_size, seed = 12)
        
    ds_size = len(df) # Define length of the dataset size
    
    train_size = int(train_split * ds_size) #Initialising the size of training set by multiplying the dataset size and percentage split of training set
    val_size = int(val_split * ds_size) 
    
    train_ds = df.take(train_size)  #Take the number of datapoints as the training dataset
    
    val_ds = df.skip(train_size).take(val_size) #Skip the number of training set and take the number of data points from the remaining data as the validation set
    test_ds = df.skip(train_size).take(val_size)   
    
    return train_ds, test_ds, val_ds

In [7]:
X_train, X_test, X_val = get_dataset_split(dataset)

print("Training Set",len(X_train))
print("Testing Set",len(X_test))
print("Validation Set",len(X_val))

Training Set 52
Testing Set 6
Validation Set 6


In [8]:
X_train = X_train.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
X_test = X_test.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
X_val = X_val.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [9]:
#Resizing and rescaling the images to get a better scale
resize_rescale = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(IMAGE_SIZE,IMAGE_SIZE),
    layers.experimental.preprocessing.Rescaling(1.0/255)
])

In [10]:
# Augmenting the data i.e, taking various angles of the data to such as horizontal flip and vertical flip so that model can 
# understand data in every possible manner. Makes the model more robust in nature 

augmentation_layer = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip('horizontal_and_vertical'),
    layers.experimental.preprocessing.RandomRotation(0.2)
])

In [11]:
# Model Building

input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = 3
model = models.Sequential([
    resize_rescale,
    augmentation_layer,
    layers.Conv2D(32, (3,3), activation = 'relu', input_shape = input_shape), # Convolution layer arg1 = kernal, arg2 = filter_size
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])

In [12]:
model.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False),
    metrics = ['accuracy']
)

In [14]:
history = model.fit(
    X_train,
    batch_size = BATCH_SIZE,
    validation_data = X_val, 
    verbose = 1,
    epochs = 50
)

Epoch 1/50
52/52 [==============================] - 30s 579ms/step - loss: 0.7341 - accuracy: 0.6454 - val_loss: 0.5590 - val_accuracy: 0.7708
Epoch 2/50
52/52 [==============================] - 28s 532ms/step - loss: 0.4955 - accuracy: 0.7752 - val_loss: 0.3714 - val_accuracy: 0.8490
Epoch 3/50
52/52 [==============================] - 29s 562ms/step - loss: 0.2854 - accuracy: 0.8834 - val_loss: 0.2654 - val_accuracy: 0.8958
Epoch 4/50
52/52 [==============================] - 28s 545ms/step - loss: 0.2145 - accuracy: 0.9141 - val_loss: 0.1843 - val_accuracy: 0.9219
Epoch 5/50
52/52 [==============================] - 28s 534ms/step - loss: 0.1758 - accuracy: 0.9357 - val_loss: 0.3504 - val_accuracy: 0.8490
Epoch 6/50
52/52 [==============================] - 29s 547ms/step - loss: 0.2286 - accuracy: 0.9111 - val_loss: 0.2755 - val_accuracy: 0.8906
Epoch 7/50
52/52 [==============================] - 29s 553ms/step - loss: 0.1608 - accuracy: 0.9339 - val_loss: 0.2541 - val_accuracy: 0.8854

In [15]:
scores = model.evaluate(X_test)


6/6 [==============================] - 2s 148ms/step - loss: 0.0646 - accuracy: 0.9844


In [16]:
history.history.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

In [17]:
acc= history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']


In [ ]:
plt.figure(figsize=(10, 8))
plt.subplot(1, 2, 1)
plt.plot(range(EPOCHS), acc, label='Training Accuracy')
plt.plot(range(EPOCHS), val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(range(EPOCHS), loss, label='Training Loss')
plt.plot(range(EPOCHS), val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()